제작자: 박찬준 <br>
소속: 고려대학교 자연어처리 & 인공지능 연구실 <br>
이메일: bcj1210@naver.com <br>
CV: https://parkchanjun.github.io/<br>

Ctranslate2란? ==> 기업에서는 속도가 생명이다 ! Transformer 모델의 인코더와 디코더를 C++로 개발하여 이를 API 형태로 서비스까지 해줄 수 있게 해주는 오픈소스<br> 

***OpenNMT pytorch와 Tensorflow 모두 Transformer 모델까지 지원*** <br>
***먼저 OpenNMT pytorch와 Tensorflow 사용법 부터 숙지하시는 것을 추천 드림***
<br>

OpenNMT Tutorial: https://github.com/Parkchanjun/OpenNMT-Colab-Tutorial <br><br>

특징은 다음과 같음 <br>

**Fast execution**
The execution aims to be faster than a general purpose deep learning framework: on standard translation tasks, it is up to 4x faster than OpenNMT-py.<br><br>
**Model quantization**
Support INT16 quantization on CPU and INT8 quantization (experimental) on CPU and GPU.<br><br>
**Parallel translation**
Translations can be run efficiently in parallel without duplicating the model data in memory.<br><br>
**Dynamic memory usage**
The memory usage changes dynamically depending on the request size while still meeting performance requirements thanks to caching allocators on both CPU and GPU.<br><br>
**Automatic instruction set dispatch**
When using Intel MKL, the dispatch to the optimal instruction set is done at runtime.<br><br>
**Ligthweight on disk**
Models can be quantized below 100MB with minimal accuracy loss. A full featured Docker image supporting GPU and CPU requires less than 1GB.<br><br>
**Easy to use translation APIs**
The project exposes translation APIs in Python and C++ to cover most integration needs.<br><br>

**1단계 : pip를 이용한 환경설정**

In [0]:
!pip install ctranslate2

**2단계: 튜토리얼용 모델 다운받기<br>**
본 튜토리얼에서는 Ctranslate2가 제공하는 데모 모델을 사용할 것이나 실제로는 직접 OpenNMT Pytorch로 나온 모델을 사용해야 할 것임.

In [0]:
!wget https://s3.amazonaws.com/opennmt-models/transformer-ende-wmt-pyOnmt.tar.gz
!tar xf transformer-ende-wmt-pyOnmt.tar.gz

**3단계: 모델을 Ctranslate2가 사용할 수 있는 형태로 변환하기**

In [0]:
!ct2-opennmt-py-converter --model_path averaged-10-epoch.pt --model_spec TransformerBase \
    --output_dir ende_ctranslate2

ende_ctranslate2 경로에 bin파일과 vocab파일이 생성된 것을 확인할 수 있다.<br>
위 예시는 OpenNMT-pytorch 기준이다.<br>

OpenNMT Tensorflow는 아래와 같이 실행하면 된다.

ct2-opennmt-tf-converter --model_path averaged-ende-export500k-v2 --model_spec TransformerBase \
    --output_dir ende_ctranslate2

**4단계: 번역하기 - Python API**

In [25]:
!pip install sentencepiece #sentencepiece 설치하기

import ctranslate2
import sentencepiece as spm

translator = ctranslate2.Translator("ende_ctranslate2/")

sp = spm.SentencePieceProcessor()
sp.Load("sentencepiece.model") #모델 로드하기 

sents=["Hello world!","hi my name is park","I want chocolate"] #예시 문장

final_outputs=[] #최종 번역 내용 담을 리스트

for sent in sents:
  pieces = sp.EncodeAsPieces(sent)
  outputs = translator.translate_batch([pieces])#번역 진행
  final_output=" ".join(outputs[0][0]['tokens']).replace("▁","") #후처리
  final_outputs.append(final_output)
  print(" ".join(outputs[0][0]['tokens']).replace("▁",""))
  
print(final_outputs)

Hallo Welt !
Hi mein Name Park ist
Ich will Schokolade machen
['Hallo Welt !', 'Hi mein Name Park ist', 'Ich will Schokolade machen']


파이썬 API 형태로 번역하는 예시이며 translator.translate_batch 에 번역할 문장을 넣어주면 된다.



**4단계: 번역하기 - C++ API**

In [0]:
#include <iostream>
#include <ctranslate2/translator.h>

int main() {
  ctranslate2::Translator translator("ende_ctranslate2/");
  ctranslate2::TranslationResult result = translator.translate({"▁H", "ello", "▁world", "!"});

  for (const auto& token : result.output())
    std::cout << token << ' ';
  std::cout << std::endl;
  return 0;
}

C++로 번역하는 예제는 위와 같다. translator.translate에 번역할 문장을 넣어주면 된다.